In [1]:
import utils
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, Lambda, LSTM, RepeatVector, Bidirectional, Masking, Dropout
from glob import glob
import os
import h5py
import time
import matplotlib
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd
import seaborn as sns
from pandas import DataFrame
import matplotlib.pyplot as plt
from matplotlib import gridspec
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.model_selection import GroupShuffleSplit


2022-09-12 20:08:43.512915: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [ ]:
filename = "dataset/N-CMAPSS_DS01-005.h5"
df_X_cols = ['alt', 'Mach', 'TRA', 'T2', 'T24', 'T30', 'T48', 'T50', 'P15', 'P2',
'P21', 'P24', 'Ps30', 'P40', 'P50', 'Nf', 'Nc', 'Wf', 'unit', 'cycle']

df_y_cols = ['fan_eff_mod', 'fan_flow_mod', 'LPC_eff_mod', 'LPC_flow_mod',
       'HPC_eff_mod', 'HPC_flow_mod', 'HPT_eff_mod', 'HPT_flow_mod',
       'LPT_eff_mod', 'LPT_flow_mod', 'unit', 'alt', 'cycle']

X = []
# y = []

df_X = pd.DataFrame(X,columns=df_X_cols)

# Load data
with h5py.File(filename, 'r') as hdf:
    # Development set
    W_dev = np.array(hdf.get('W_dev'))             # W
    X_s_dev = np.array(hdf.get('X_s_dev'))         # X_s
#         X_v_dev = np.array(hdf.get('X_v_dev'))         # X_v
#         T_dev = np.array(hdf.get('T_dev'))             # T
    Y_dev = np.array(hdf.get('Y_dev'))             # RUL  
    A_dev = np.array(hdf.get('A_dev'))             # Auxiliary

    # Test set
    W_test = np.array(hdf.get('W_test'))           # W
    X_s_test = np.array(hdf.get('X_s_test'))       # X_s
#         X_v_test = np.array(hdf.get('X_v_test'))       # X_v
#         T_test = np.array(hdf.get('T_test'))           # T
    Y_test = np.array(hdf.get('Y_test'))           # RUL  
    A_test = np.array(hdf.get('A_test'))           # Auxiliary

    # Varnams
    W_var = np.array(hdf.get('W_var'))
    X_s_var = np.array(hdf.get('X_s_var'))  
#         X_v_var = np.array(hdf.get('X_v_var')) 
#         T_var = np.array(hdf.get('T_var'))
    A_var = np.array(hdf.get('A_var'))

    # from np.array to list dtype U4/U5
    W_var = list(np.array(W_var, dtype='U20'))
    X_s_var = list(np.array(X_s_var, dtype='U20'))  
#         X_v_var = list(np.array(X_v_var, dtype='U20')) 
#         T_var = list(np.array(T_var, dtype='U20'))
    A_var = list(np.array(A_var, dtype='U20'))
    W_Xs_var = W_var + X_s_var
W_Xs_dev = np.concatenate((W_dev, X_s_dev), axis=1)
W_Xs_test = np.concatenate((W_test, X_s_test), axis=1)

df_A_dev = DataFrame(data=A_dev, columns=A_var)
df_A_test = DataFrame(data=A_test, columns=A_var)

df_W_Xs_dev = DataFrame(data=W_Xs_dev, columns=W_Xs_var)
df_W_Xs_test = DataFrame(data=W_Xs_test, columns=W_Xs_var)
#     df_T = DataFrame(data=T, columns=T_var)

# Add the column "unit" to all dataframes
#     df_T['unit'] = df_A['unit'].values
#     df_T['alt'] = df_W_Xs['alt'].values
df_W_Xs_dev['unit'] = df_A_dev['unit'].values
df_W_Xs_test['unit'] = df_A_test['unit'].values

#     df_T['cycle'] = df_A['cycle'].values
df_W_Xs_dev['cycle'] = df_A_dev['cycle'].values
df_W_Xs_test['cycle'] = df_A_test['cycle'].values

df_W_Xs_dev['Fc'] = df_A_dev['Fc'].values
df_W_Xs_test['Fc'] = df_A_test['Fc'].values
#     df_W_Xs['HS'] = df_A['hs'].values

cols = ['unit', 'cycle', 'Fc', 'alt', 'Mach',  'TRA', 'T2', 'T24', 'T30', 'T48', 'T50', 'P15', 'P2', 'P21', 'P24', 'Ps30', 'P40', 'P50', 'Nf', 'Nc', 'Wf', ]
df_W_Xs_dev = df_W_Xs_dev[cols]
df_W_Xs_test = df_W_Xs_test[cols]

for col in df_W_Xs_dev:
    if col in ['unit', 'cycle', 'alt', 'Fc']:
        df_W_Xs_dev[col] = df_W_Xs_dev[col].astype(int)
        df_W_Xs_test[col] = df_W_Xs_test[col].astype(int)
    else:
        df_W_Xs_dev[col] = df_W_Xs_dev[col].astype(float)
        df_W_Xs_test[col] = df_W_Xs_test[col].astype(float)

        


In [ ]:
df_W_Xs_test

In [ ]:
print(df_W_Xs_dev.shape, Y_dev.shape)
print(df_W_Xs_test.shape, Y_test.shape)

In [ ]:
df_y_dev = pd.DataFrame(zip(df_W_Xs_dev.unit, Y_dev.reshape(len(Y_dev))), columns=['unit', 'RUL'])
df_y_test = pd.DataFrame(zip(df_W_Xs_test.unit, Y_test.reshape(len(Y_test))), columns=['unit', 'RUL'])

In [ ]:
def add_operating_condition(df):
    df_op_cond = df.copy()
    # 'Fc', 'alt', 'Mach',  'TRA'
    df_op_cond['TRA'] = abs(df_op_cond['TRA'].round()).astype(int)
#     df_op_cond['Mach'] = abs(df_op_cond['Mach'].round(decimals=2))

    # converting settings to string and concatanating makes the operating condition into a categorical variable
    df_op_cond['op_cond'] = df_op_cond['Fc'].astype(str) + '_' + \
                            df_op_cond['Mach'].round(decimals=1).astype(str) + '_' + \
                            df_op_cond['TRA'].astype(str)
#     (df_op_cond['alt']//1000).astype(str) + '_' + \
    df_op_cond['op_cond'] = df_op_cond.op_cond.astype(str)

    return df_op_cond

def condition_scaler(df, sensor_names):
    # apply operating condition specific scaling
    scaler = StandardScaler()
    for condition in df['op_cond'].unique():
        print(list(df['op_cond'].unique()).index(condition))
#         scaler.fit(df.loc[df['op_cond'] == condition, sensor_names])
        df.loc[df['op_cond'] == condition, sensor_names] = scaler.fit_transform(df.loc[df['op_cond'] == condition, sensor_names])
    return df

def exponential_smoothing(df, sensors, n_samples, alpha=0.4):
    df = df.copy()
    # first, take the exponential weighted mean
    df[sensors] = df.groupby('unit')[sensors].apply(lambda x: x.ewm(alpha=alpha).mean()).reset_index(level=0, drop=True)

    # second, drop first n_samples of each unit_nr to reduce filter delay
    def create_mask(data, samples):
        result = np.ones_like(data)
        result[0:samples] = 0
        return result

    mask = df.groupby('unit')['unit'].transform(create_mask, samples=n_samples).astype(bool)
    df = df[mask]

    return df

In [ ]:
sensor_names = ['T2', 'T24', 'T30', 'T48', 'T50', 'P15', 'P2', 'P21', 'P24', 'Ps30', 'P40', 'P50', 'Nf', 'Nc', 'Wf',]
sensors = ['T2', 'T24', 'T30', 'T48', 'T50', 'P15', 'P2', 'P21', 'P24', 'Ps30', 'P40', 'P50', 'Nf', 'Nc', 'Wf',]
# remove unused sensors
drop_sensors = [element for element in sensor_names if element not in sensors]


In [ ]:
drop_sensors

In [ ]:
X_train_pre = add_operating_condition(df_W_Xs_dev.drop(drop_sensors, axis=1))
X_test_pre = add_operating_condition(df_W_Xs_test.drop(drop_sensors, axis=1))

In [ ]:
X_train_pre.op_cond.unique()

In [ ]:
len(X_train_pre.op_cond.unique())

In [ ]:
X_train_pre.info()

In [ ]:
X_train_pre = condition_scaler(X_train_pre, sensors)
X_test_pre = condition_scaler(X_test_pre, sensors)
print(X_train_pre.shape)

X_tr_pre_copy = X_train_pre.copy()
X_te_pre_copy = X_test_pre.copy()

In [ ]:
X_train_pre = exponential_smoothing(X_train_pre, sensors, 0, alpha=0.1)
X_test_pre = exponential_smoothing(X_test_pre, sensors, 0, alpha=0.1)

In [ ]:
X_train_pre

In [ ]:
def gen_train_data(df, sequence_length, columns):
    data = df[columns].values
    num_elements = data.shape[0]

    # -1 and +1 because of Python indexing
    for start, stop in zip(range(0, num_elements - (sequence_length - 1)), range(sequence_length, num_elements + 1)):
        yield data[start:stop, :]


def gen_data_wrapper(df, sequence_length, columns, unit_nrs=np.array([])):
    if unit_nrs.size <= 0:
        unit_nrs = df['unit'].unique()

    data_gen = (list(gen_train_data(df[df['unit'] == unit_nr], sequence_length, columns))
                for unit_nr in unit_nrs)
    data_array = np.concatenate(list(data_gen)).astype(np.float32)
    return data_array


def gen_labels(df, sequence_length, label):
    data_matrix = df[label].values
    num_elements = data_matrix.shape[0]

    # -1 because I want to predict the rul of that last row in the sequence, not the next row
    return data_matrix[sequence_length - 1:num_elements, :]


def gen_label_wrapper(df, sequence_length, label, unit_nrs=np.array([])):
    if unit_nrs.size <= 0:
        unit_nrs = df['unit'].unique()

    label_gen = [gen_labels(df[df['unit'] == unit_nr], sequence_length, label)
                 for unit_nr in unit_nrs]
    label_array = np.concatenate(label_gen).astype(np.float32)
    return label_array


In [ ]:
sequence_length = 30

In [ ]:
gss = GroupShuffleSplit(n_splits=1, train_size=0.87, random_state=42)
for train_unit, val_unit in gss.split(X_train_pre['unit'].unique(), groups=X_train_pre['unit'].unique()):
    print(train_unit, val_unit)

In [ ]:
# train-val split
gss = GroupShuffleSplit(n_splits=1, train_size=0.87, random_state=42)
# generate the train/val for *each* sample -> for that we iterate over the train and val units we want
# this is a for that iterates only once and in that iterations at the same time iterates over all the values we want,
# i.e. train_unit and val_unit are not a single value but a set of training/vali units
for train_unit, val_unit in gss.split(X_train_pre['unit'].unique(), groups=X_train_pre['unit'].unique()):
    train_unit = X_train_pre['unit'].unique()[train_unit]  # gss returns indexes and index starts at 1
    val_unit = X_train_pre['unit'].unique()[val_unit]

    x_train = gen_data_wrapper(X_train_pre, sequence_length, sensors, train_unit)
    y_train = gen_label_wrapper(df_y_dev, sequence_length, ['RUL'], train_unit)

    x_val = gen_data_wrapper(X_train_pre, sequence_length, sensors, val_unit)
    y_val = gen_label_wrapper(df_y_dev, sequence_length, ['RUL'], val_unit)


In [ ]:
def gen_test_data(df, sequence_length, columns, mask_value):
    if df.shape[0] < sequence_length:
        data_matrix = np.full(shape=(sequence_length, len(columns)), fill_value=mask_value)  # pad
        idx = data_matrix.shape[0] - df.shape[0]
        data_matrix[idx:, :] = df[columns].values  # fill with available data
    else:
        data_matrix = df[columns].values

    # specifically yield the last possible sequence
    stop = data_matrix.shape[0]
    start = stop - sequence_length
    for i in list(range(1)):
        yield data_matrix[start:stop, :]

In [ ]:
# create sequences for test
test_gen = (list(gen_test_data(X_test_pre[X_test_pre['unit'] == unit_nr], sequence_length, sensors, -99.))
            for unit_nr in X_test_pre['unit'].unique())

In [ ]:
x_test = np.concatenate(list(test_gen)).astype(np.float32)

In [ ]:
x_train, y_train, x_val, y_val, x_test, df_y_test.RUL

In [ ]:
y_val.shape

In [ ]:
x_train.shape

In [ ]:
x_val.shape

In [ ]:
y_train.shape

In [ ]:
y_val.shape

In [ ]:
x_test.shape

In [ ]:
df_y_test.RUL.shape

os.mkdir('./RULRVE_numpydata')

np.save('RULRVE_numpydata/x_train.npy', x_train)
np.save('RULRVE_numpydata/x_val.npy', x_val)
np.save('RULRVE_numpydata/y_train.npy', y_train)
np.save('RULRVE_numpydata/y_val.npy', y_val)
np.save('RULRVE_numpydata/x_test.npy', x_test)

df_y_test.to_csv('RULRVE_numpydata/df_y_test.csv')

In [ ]:
from os.path import join as opj
def loader(filepath):
    with open(filepath, 'rb') as fp:
        np_ndarray = np.load(fp)
    print("{} has a numpy ndarray of shape: {}".format(filepath, np_ndarray.shape))
    return np_ndarray

data_dir = '/data/adas_vision_data1/users/adithya/turbofan_ncmapss/RULRVE_numpydata'
x_train = loader(opj(data_dir, 'x_train.npy'))
x_val = loader(opj(data_dir, 'x_val.npy'))
y_train = loader(opj(data_dir, 'y_train.npy'))
y_val = loader(opj(data_dir, 'y_val.npy'))
# x_val = x_val[:300000, ...]
# y_val = y_val[:300000, ...]
x_test = loader(opj(data_dir, 'x_test.npy'))
df_y_test = pd.read_csv(opj(data_dir, 'df_y_test.csv'))
y_test = df_y_test.RUL

In [ ]:
os.mkdir('./RULRVE_numpydata_1_val_eng')

np.save('RULRVE_numpydata_1_val_eng/x_train.npy', x_train)
np.save('RULRVE_numpydata_1_val_eng/x_val.npy', x_val)
np.save('RULRVE_numpydata_1_val_eng/y_train.npy', y_train)
np.save('RULRVE_numpydata_1_val_eng/y_val.npy', y_val)
np.save('RULRVE_numpydata_1_val_eng/x_test.npy', x_test)

df_y_test.to_csv('RULRVE_numpydata_1_val_eng/df_y_test.csv')

In [ ]:
x_val

In [ ]:
66: 4369/4369 - 184s - loss: 13.2013 - kl_loss: 3.5793 - reg_loss: 9.6220 - val_loss: 12.9760 - val_kl_loss: 4.5276 - val_reg_loss: 8.4484 - 184s/epoch - 42ms/step

In [ ]:
import sys
sys.path.append('/home/a0484689/PycharmProjects/Remaining-Useful-Life-Estimation-Variational')

In [ ]:
import utils
import model

In [ ]:
# ----------------------------- MODEL -----------------------------------
# no_of_samples = x_train.shape[0]
timesteps = x_train.shape[1]
input_dim = x_train.shape[2]
intermediate_dim = 300
batch_size = 128
latent_dim = 2
epochs = 10000
optimizer = 'adam'

In [ ]:
RVE = model.create_model(timesteps, input_dim, intermediate_dim, 
                         batch_size, latent_dim, epochs, optimizer,)

In [ ]:
# Callbacks for training
model_callbacks = utils.get_callbacks(RVE, x_train, y_train)

In [ ]:
# --------------------------- TRAINING ---------------------------------
results = RVE.fit(x_train, y_train, shuffle=True, epochs=epochs,
                  batch_size=batch_size, validation_data= (x_val, y_val),
                  callbacks=model_callbacks, verbose=2)
# -----------------------------------------------------------------------


In [ ]:
def load_data(filename):
    df_X_cols = ['alt', 'Mach', 'TRA', 'T2', 'T24', 'T30', 'T48', 'T50', 'P15', 'P2',
                 'P21', 'P24', 'Ps30', 'P40', 'P50', 'Nf', 'Nc', 'Wf', 'unit', 'cycle']

    df_y_cols = ['fan_eff_mod', 'fan_flow_mod', 'LPC_eff_mod', 'LPC_flow_mod',
                 'HPC_eff_mod', 'HPC_flow_mod', 'HPT_eff_mod', 'HPT_flow_mod',
                 'LPT_eff_mod', 'LPT_flow_mod', 'unit', 'alt', 'cycle']

    X = []
    # y = []

    df_X = pd.DataFrame(X, columns=df_X_cols)

    # Load data
    with h5py.File(filename, 'r') as hdf:
        # Development set
        W_dev = np.array(hdf.get('W_dev'))  # W
        X_s_dev = np.array(hdf.get('X_s_dev'))  # X_s
        #         X_v_dev = np.array(hdf.get('X_v_dev'))         # X_v
        #         T_dev = np.array(hdf.get('T_dev'))             # T
        Y_dev = np.array(hdf.get('Y_dev'))  # RUL
        A_dev = np.array(hdf.get('A_dev'))  # Auxiliary

        # Test set
        W_test = np.array(hdf.get('W_test'))  # W
        X_s_test = np.array(hdf.get('X_s_test'))  # X_s
        #         X_v_test = np.array(hdf.get('X_v_test'))       # X_v
        #         T_test = np.array(hdf.get('T_test'))           # T
        Y_test = np.array(hdf.get('Y_test'))  # RUL
        A_test = np.array(hdf.get('A_test'))  # Auxiliary

        # Varnams
        W_var = np.array(hdf.get('W_var'))
        X_s_var = np.array(hdf.get('X_s_var'))
        #         X_v_var = np.array(hdf.get('X_v_var'))
        #         T_var = np.array(hdf.get('T_var'))
        A_var = np.array(hdf.get('A_var'))

        # from np.array to list dtype U4/U5
        W_var = list(np.array(W_var, dtype='U20'))
        X_s_var = list(np.array(X_s_var, dtype='U20'))
        #         X_v_var = list(np.array(X_v_var, dtype='U20'))
        #         T_var = list(np.array(T_var, dtype='U20'))
        A_var = list(np.array(A_var, dtype='U20'))
        W_Xs_var = W_var + X_s_var
    W_Xs_dev = np.concatenate((W_dev, X_s_dev), axis=1)
    W_Xs_test = np.concatenate((W_test, X_s_test), axis=1)

    df_A_dev = DataFrame(data=A_dev, columns=A_var)
    df_A_test = DataFrame(data=A_test, columns=A_var)

    df_W_Xs_dev = DataFrame(data=W_Xs_dev, columns=W_Xs_var)
    df_W_Xs_test = DataFrame(data=W_Xs_test, columns=W_Xs_var)
    #     df_T = DataFrame(data=T, columns=T_var)

    # Add the column "unit" to all dataframes
    #     df_T['unit'] = df_A['unit'].values
    #     df_T['alt'] = df_W_Xs['alt'].values
    df_W_Xs_dev['unit'] = df_A_dev['unit'].values
    df_W_Xs_test['unit'] = df_A_test['unit'].values

    #     df_T['cycle'] = df_A['cycle'].values
    df_W_Xs_dev['cycle'] = df_A_dev['cycle'].values
    df_W_Xs_test['cycle'] = df_A_test['cycle'].values

    df_W_Xs_dev['Fc'] = df_A_dev['Fc'].values
    df_W_Xs_test['Fc'] = df_A_test['Fc'].values
    #     df_W_Xs['HS'] = df_A['hs'].values

    cols = ['unit', 'cycle', 'Fc', 'alt', 'Mach', 'TRA', 'T2', 'T24', 'T30', 'T48', 'T50', 'P15', 'P2', 'P21', 'P24',
            'Ps30', 'P40', 'P50', 'Nf', 'Nc', 'Wf', ]
    df_W_Xs_dev = df_W_Xs_dev[cols]
    df_W_Xs_test = df_W_Xs_test[cols]

    for col in df_W_Xs_dev:
        if col in ['unit', 'cycle', 'alt', 'Fc']:
            df_W_Xs_dev[col] = df_W_Xs_dev[col].astype(int)
            df_W_Xs_test[col] = df_W_Xs_test[col].astype(int)
        else:
            df_W_Xs_dev[col] = df_W_Xs_dev[col].astype(float)
            df_W_Xs_test[col] = df_W_Xs_test[col].astype(float)

    df_y_dev = pd.DataFrame(zip(df_W_Xs_dev.unit, Y_dev.reshape(len(Y_dev))), columns=['unit', 'RUL'])
    df_y_test = pd.DataFrame(zip(df_W_Xs_test.unit, Y_test.reshape(len(Y_test))), columns=['unit', 'RUL'])

    return df_W_Xs_dev, df_y_dev, df_W_Xs_test, df_y_test

In [ ]:
filename1 = "/data/adas_vision_data1/users/adithya/turbofan_ncmapss/dataset/N-CMAPSS_DS01-005.h5"
filename2 = "/data/adas_vision_data1/users/adithya/turbofan_ncmapss/dataset/N-CMAPSS_DS02-006.h5"
filename3 = "/data/adas_vision_data1/users/adithya/turbofan_ncmapss/dataset/N-CMAPSS_DS03-012.h5"
filename4 = "/data/adas_vision_data1/users/adithya/turbofan_ncmapss/dataset/N-CMAPSS_DS04.h5"
filename5 = "/data/adas_vision_data1/users/adithya/turbofan_ncmapss/dataset/N-CMAPSS_DS05.h5"
filename6 = "/data/adas_vision_data1/users/adithya/turbofan_ncmapss/dataset/N-CMAPSS_DS06.h5"
filename7 = "/data/adas_vision_data1/users/adithya/turbofan_ncmapss/dataset/N-CMAPSS_DS07.h5"
filename8 = "/data/adas_vision_data1/users/adithya/turbofan_ncmapss/dataset/N-CMAPSS_DS08a-009.h5"
filename9 = "/data/adas_vision_data1/users/adithya/turbofan_ncmapss/dataset/N-CMAPSS_DS08c-008.h5"


In [ ]:
# df_W_Xs_dev1, df_y_dev1, df_W_Xs_test1, df_y_test1 = load_data(filename1)
# df_W_Xs_dev2, df_y_dev2, df_W_Xs_test2, df_y_test2 = load_data(filename2)
# df_W_Xs_dev3, df_y_dev3, df_W_Xs_test3, df_y_test3 = load_data(filename3)
# df_W_Xs_dev4, df_y_dev4, df_W_Xs_test4, df_y_test4 = load_data(filename4)
# df_W_Xs_dev5, df_y_dev5, df_W_Xs_test5, df_y_test5 = load_data(filename5)
# df_W_Xs_dev6, df_y_dev6, df_W_Xs_test6, df_y_test6 = load_data(filename6)
# df_W_Xs_dev7, df_y_dev7, df_W_Xs_test7, df_y_test7 = load_data(filename7)
# df_W_Xs_dev8, df_y_dev8, df_W_Xs_test8, df_y_test8 = load_data(filename8)
# df_W_Xs_dev9, df_y_dev9, df_W_Xs_test9, df_y_test9 = load_data(filename9)

df_W_Xs_dev1.to_pickle('df_W_Xs_dev1.pkl')
df_y_dev1.to_pickle('df_y_dev1.pkl')
df_W_Xs_test1.to_pickle('df_W_Xs_test1.pkl')
df_y_test1.to_pickle('df_y_test1.pkl')

df_W_Xs_dev2.to_pickle('df_W_Xs_dev2.pkl')
df_y_dev2.to_pickle('df_y_dev2.pkl')
df_W_Xs_test2.to_pickle('df_W_Xs_test2.pkl')
df_y_test2.to_pickle('df_y_test2.pkl')

df_W_Xs_dev3.to_pickle('df_W_Xs_dev3.pkl')
df_y_dev3.to_pickle('df_y_dev3.pkl')
df_W_Xs_test3.to_pickle('df_W_Xs_test3.pkl')
df_y_test3.to_pickle('df_y_test3.pkl')

df_W_Xs_dev4.to_pickle('df_W_Xs_dev4.pkl')
df_y_dev4.to_pickle('df_y_dev4.pkl')
df_W_Xs_test4.to_pickle('df_W_Xs_test4.pkl')
df_y_test4.to_pickle('df_y_test4.pkl')

df_W_Xs_dev5.to_pickle('df_W_Xs_dev5.pkl')
df_y_dev5.to_pickle('df_y_dev5.pkl')
df_W_Xs_test5.to_pickle('df_W_Xs_test5.pkl')
df_y_test5.to_pickle('df_y_test5.pkl')

df_W_Xs_dev6.to_pickle('df_W_Xs_dev6.pkl')
df_y_dev6.to_pickle('df_y_dev6.pkl')
df_W_Xs_test6.to_pickle('df_W_Xs_test6.pkl')
df_y_test6.to_pickle('df_y_test6.pkl')

df_W_Xs_dev7.to_pickle('df_W_Xs_dev7.pkl')
df_y_dev7.to_pickle('df_y_dev7.pkl')
df_W_Xs_test7.to_pickle('df_W_Xs_test7.pkl')
df_y_test7.to_pickle('df_y_test7.pkl')

df_W_Xs_dev8.to_pickle('df_W_Xs_dev8.pkl')
df_y_dev8.to_pickle('df_y_dev8.pkl')
df_W_Xs_test8.to_pickle('df_W_Xs_test8.pkl')
df_y_test8.to_pickle('df_y_test8.pkl')

df_W_Xs_dev9.to_pickle('df_W_Xs_dev9.pkl')
df_y_dev9.to_pickle('df_y_dev9.pkl')
df_W_Xs_test9.to_pickle('df_W_Xs_test9.pkl')
df_y_test9.to_pickle('df_y_test9.pkl')


In [ ]:
df_W_Xs_dev1 = pd.read_pickle('df_W_Xs_dev1.pkl')
df_y_dev1 = pd.read_pickle('df_y_dev1.pkl')
df_W_Xs_test1 = pd.read_pickle('df_W_Xs_test1.pkl')
df_y_test1 = pd.read_pickle('df_y_test1.pkl')

# df_W_Xs_dev2 = pd.read_pickle('df_W_Xs_dev2.pkl')
# df_y_dev2 = pd.read_pickle('df_y_dev2.pkl')
# df_W_Xs_test2 = pd.read_pickle('df_W_Xs_test2.pkl')
# df_y_test2 = pd.read_pickle('df_y_test2.pkl')

# df_W_Xs_dev3 = pd.read_pickle('df_W_Xs_dev3.pkl')
# df_y_dev3 = pd.read_pickle('df_y_dev3.pkl')
# df_W_Xs_test3 = pd.read_pickle('df_W_Xs_test3.pkl')
# df_y_test3 = pd.read_pickle('df_y_test3.pkl')

# df_W_Xs_dev4 = pd.read_pickle('df_W_Xs_dev4.pkl')
# df_y_dev4 = pd.read_pickle('df_y_dev4.pkl')
# df_W_Xs_test4 = pd.read_pickle('df_W_Xs_test4.pkl')
# df_y_test4 = pd.read_pickle('df_y_test4.pkl')

# df_W_Xs_dev5 = pd.read_pickle('df_W_Xs_dev5.pkl')
# df_y_dev5 = pd.read_pickle('df_y_dev5.pkl')
# df_W_Xs_test5 = pd.read_pickle('df_W_Xs_test5.pkl')
# df_y_test5 = pd.read_pickle('df_y_test5.pkl')

# df_W_Xs_dev6 = pd.read_pickle('df_W_Xs_dev6.pkl')
# df_y_dev6 = pd.read_pickle('df_y_dev6.pkl')
# df_W_Xs_test6 = pd.read_pickle('df_W_Xs_test6.pkl')
# df_y_test6 = pd.read_pickle('df_y_test6.pkl')

# df_W_Xs_dev7 = pd.read_pickle('df_W_Xs_dev7.pkl')
# df_y_dev7 = pd.read_pickle('df_y_dev7.pkl')
# df_W_Xs_test7 = pd.read_pickle('df_W_Xs_test7.pkl')
# df_y_test7 = pd.read_pickle('df_y_test7.pkl')

# df_W_Xs_dev8 = pd.read_pickle('df_W_Xs_dev8.pkl')
# df_y_dev8 = pd.read_pickle('df_y_dev8.pkl')
# df_W_Xs_test8 = pd.read_pickle('df_W_Xs_test8.pkl')
# df_y_test8 = pd.read_pickle('df_y_test8.pkl')

# df_W_Xs_dev9 = pd.read_pickle('df_W_Xs_dev9.pkl')
# df_y_dev9 = pd.read_pickle('df_y_dev9.pkl')
# df_W_Xs_test9 = pd.read_pickle('df_W_Xs_test9.pkl')
# df_y_test9 = pd.read_pickle('df_y_test9.pkl')


In [ ]:
sequence_length = 30
sensors = ['T2', 'T24', 'T30', 'T48', 'T50', 'P15', 'P2', 'P21', 'P24', 'Ps30', 'P40', 'P50', 'Nf', 'Nc', 'Wf', ]

In [ ]:
def prepare_data(sequence_length, sensors, df_W_Xs_dev, df_y_dev, df_W_Xs_test, df_y_test):
    # remove unused sensors
    sensor_names = ['T2', 'T24', 'T30', 'T48', 'T50', 'P15', 'P2', 'P21', 'P24', 'Ps30', 'P40', 'P50', 'Nf', 'Nc',
                    'Wf', ]
    drop_sensors = [element for element in sensor_names if element not in sensors]

    X_train_pre = add_operating_condition(df_W_Xs_dev.drop(drop_sensors, axis=1))
    X_test_pre = add_operating_condition(df_W_Xs_test.drop(drop_sensors, axis=1))

    X_train_pre = condition_scaler(X_train_pre, sensors)
    X_test_pre = condition_scaler(X_test_pre, sensors)

    X_train_pre = exponential_smoothing(X_train_pre, sensors, 0, alpha=0.1)
    X_test_pre = exponential_smoothing(X_test_pre, sensors, 0, alpha=0.1)

    # train-val split
    gss = GroupShuffleSplit(n_splits=1, train_size=0.87, random_state=42)
    # generate the train/val for *each* sample -> for that we iterate over the train and val units we want
    # this is a for that iterates only once and in that iterations at the same time iterates over all the values we want,
    # i.e. train_unit and val_unit are not a single value but a set of training/vali units
    for train_unit, val_unit in gss.split(X_train_pre['unit'].unique(), groups=X_train_pre['unit'].unique()):
        train_unit = X_train_pre['unit'].unique()[train_unit]  # gss returns indexes and index starts at 1
        val_unit = X_train_pre['unit'].unique()[val_unit]

        x_train = gen_data_wrapper(X_train_pre, sequence_length, sensors, train_unit)
        y_train = gen_label_wrapper(df_y_dev, sequence_length, ['RUL'], train_unit)

        x_val = gen_data_wrapper(X_train_pre, sequence_length, sensors, val_unit)
        y_val = gen_label_wrapper(df_y_dev, sequence_length, ['RUL'], val_unit)

#     # create sequences for test
#     test_gen = (list(gen_test_data(X_test_pre[X_test_pre['unit'] == unit_nr], sequence_length, sensors, -99.))
#                 for unit_nr in X_test_pre['unit'].unique())
#     x_test = np.concatenate(list(test_gen)).astype(np.float32)
    for test_unit in X_test_pre.unit.unique():
        x_test = gen_data_wrapper(X_test_pre, sequence_length, sensors, test_unit)
        y_test = gen_label_wrapper(df_y_test, sequence_length, ['RUL'], test_unit)

    return x_train, y_train, x_val, y_val, x_test, y_test



In [ ]:
import sys
sys.path.append('/home/a0484689/PycharmProjects/Remaining-Useful-Life-Estimation-Variational')
from ncmapss_rul_rve import *

In [ ]:
df_W_Xs_test1.unit.unique()

In [ ]:
'''

This module is to regenerate x_test and y_test in the same format as train data.
The previous module has already been updated

'''
def prepare_data(sequence_length, sensors, df_W_Xs_test, df_y_test):
    # remove unused sensors
    sensor_names = ['T2', 'T24', 'T30', 'T48', 'T50', 'P15', 'P2', 'P21', 'P24', 'Ps30', 'P40', 'P50', 'Nf', 'Nc',
                    'Wf', ]
    drop_sensors = [element for element in sensor_names if element not in sensors]

#     X_train_pre = add_operating_condition(df_W_Xs_dev.drop(drop_sensors, axis=1))
    X_test_pre = add_operating_condition(df_W_Xs_test.drop(drop_sensors, axis=1))

#     X_train_pre = condition_scaler(X_train_pre, sensors)
    X_test_pre = condition_scaler(X_test_pre, sensors)

#     X_train_pre = exponential_smoothing(X_train_pre, sensors, 0, alpha=0.1)
    X_test_pre = exponential_smoothing(X_test_pre, sensors, 0, alpha=0.1)

    # train-val split
#     gss = GroupShuffleSplit(n_splits=1, train_size=0.87, random_state=42)
#     # generate the train/val for *each* sample -> for that we iterate over the train and val units we want
#     # this is a for that iterates only once and in that iterations at the same time iterates over all the values we want,
#     # i.e. train_unit and val_unit are not a single value but a set of training/vali units
#     for train_unit, val_unit in gss.split(X_train_pre['unit'].unique(), groups=X_train_pre['unit'].unique()):
#         train_unit = X_train_pre['unit'].unique()[train_unit]  # gss returns indexes and index starts at 1
#         val_unit = X_train_pre['unit'].unique()[val_unit]

#         x_train = gen_data_wrapper(X_train_pre, sequence_length, sensors, train_unit)
#         y_train = gen_label_wrapper(df_y_dev, sequence_length, ['RUL'], train_unit)

#         x_val = gen_data_wrapper(X_train_pre, sequence_length, sensors, val_unit)
#         y_val = gen_label_wrapper(df_y_dev, sequence_length, ['RUL'], val_unit)

#     # create sequences for test
#     test_gen = (list(gen_test_data(X_test_pre[X_test_pre['unit'] == unit_nr], sequence_length, sensors, -99.))
#                 for unit_nr in X_test_pre['unit'].unique())
#     x_test = np.concatenate(list(test_gen)).astype(np.float32)
#     for test_unit in X_test_pre.unit.unique():
    x_test = gen_data_wrapper(X_test_pre, sequence_length, sensors, X_test_pre.unit.unique())
    y_test = gen_label_wrapper(df_y_test, sequence_length, ['RUL'], X_test_pre.unit.unique())

    return x_test, y_test

sequence_length = 30
sensors = ['T2', 'T24', 'T30', 'T48', 'T50', 'P15', 'P2', 'P21', 'P24', 'Ps30', 'P40', 'P50', 'Nf', 'Nc', 'Wf', ]

'''Read from dataframe'''
# df_W_Xs_test1 = pd.read_pickle('df_W_Xs_test1.pkl')
# df_y_test1 = pd.read_pickle('df_y_test1.pkl')

# df_W_Xs_test2 = pd.read_pickle('df_W_Xs_test2.pkl')
# df_y_test2 = pd.read_pickle('df_y_test2.pkl')

# df_W_Xs_test3 = pd.read_pickle('df_W_Xs_test3.pkl')
# df_y_test3 = pd.read_pickle('df_y_test3.pkl')

# df_W_Xs_test4 = pd.read_pickle('df_W_Xs_test4.pkl')
# df_y_test4 = pd.read_pickle('df_y_test4.pkl')

df_W_Xs_test5 = pd.read_pickle('df_W_Xs_test5.pkl')
df_y_test5 = pd.read_pickle('df_y_test5.pkl')

# df_W_Xs_test6 = pd.read_pickle('df_W_Xs_test6.pkl')
# df_y_test6 = pd.read_pickle('df_y_test6.pkl')

# df_W_Xs_test7 = pd.read_pickle('df_W_Xs_test7.pkl')
# df_y_test7 = pd.read_pickle('df_y_test7.pkl')

# df_W_Xs_test8 = pd.read_pickle('df_W_Xs_test8.pkl')
# df_y_test8 = pd.read_pickle('df_y_test8.pkl')

# df_W_Xs_test9 = pd.read_pickle('df_W_Xs_test9.pkl')
# df_y_test9 = pd.read_pickle('df_y_test9.pkl')

'''Prepare Numpy arrays'''
# x_test1, y_test1 = prepare_data(sequence_length, sensors, df_W_Xs_test1, df_y_test1)
# x_test2, y_test2 = prepare_data(sequence_length, sensors, df_W_Xs_test2, df_y_test2)
# x_test3, y_test3 = prepare_data(sequence_length, sensors, df_W_Xs_test3, df_y_test3)
# x_test4, y_test4 = prepare_data(sequence_length, sensors, df_W_Xs_test4, df_y_test4)
x_test5, y_test5 = prepare_data(sequence_length, sensors, df_W_Xs_test5, df_y_test5)

# x_test6, y_test6 = prepare_data(sequence_length, sensors, df_W_Xs_test6, df_y_test6)
# x_test7, y_test7 = prepare_data(sequence_length, sensors, df_W_Xs_test7, df_y_test7)
# x_test8, y_test8 = prepare_data(sequence_length, sensors, df_W_Xs_test8, df_y_test8)
# x_test9, y_test9 = prepare_data(sequence_length, sensors, df_W_Xs_test9, df_y_test9)

'''Save numpy arrays'''
# np.save('RULRVE_numpydata_total/x_test1_total.npy', x_test1)
# np.save('RULRVE_numpydata_total/y_test1_total.npy', y_test1)

# np.save('RULRVE_numpydata_total/x_test2_total.npy', x_test2)
# np.save('RULRVE_numpydata_total/y_test2_total.npy', y_test2)

# np.save('RULRVE_numpydata_total/x_test3_total.npy', x_test3)
# np.save('RULRVE_numpydata_total/y_test3_total.npy', y_test3)

# np.save('RULRVE_numpydata_total/x_test4_total.npy', x_test4)
# np.save('RULRVE_numpydata_total/y_test4_total.npy', y_test4)

np.save('RULRVE_numpydata_total/x_test5_total.npy', x_test5)
np.save('RULRVE_numpydata_total/y_test5_total.npy', y_test5)

# np.save('RULRVE_numpydata_total/x_test6_total.npy', x_test6)
# np.save('RULRVE_numpydata_total/y_test6_total.npy', y_test6)

# np.save('RULRVE_numpydata_total/x_test7_total.npy', x_test7)
# np.save('RULRVE_numpydata_total/y_test7_total.npy', y_test7)

# np.save('RULRVE_numpydata_total/x_test8_total.npy', x_test8)
# np.save('RULRVE_numpydata_total/y_test8_total.npy', y_test8)

# np.save('RULRVE_numpydata_total/x_test9_total.npy', x_test9)
# np.save('RULRVE_numpydata_total/y_test9_total.npy', y_test9)


In [ ]:
x_test1.shape

In [ ]:
y_test1.shape

In [ ]:
# x_train1, y_train1, x_val1, y_val1, x_test1, y_test1 = prepare_data(sequence_length, sensors, df_W_Xs_dev1, df_y_dev1, df_W_Xs_test1, df_y_test1)
# x_train2, y_train2, x_val2, y_val2, x_test2, y_test2 = prepare_data(sequence_length, sensors, df_W_Xs_dev2, df_y_dev2, df_W_Xs_test2, df_y_test2)
# x_train3, y_train3, x_val3, y_val3, x_test3, y_test3 = prepare_data(sequence_length, sensors, df_W_Xs_dev3, df_y_dev3, df_W_Xs_test3, df_y_test3)
# x_train4, y_train4, x_val4, y_val4, x_test4, y_test4 = prepare_data(sequence_length, sensors, df_W_Xs_dev4, df_y_dev4, df_W_Xs_test4, df_y_test4)
# x_train5, y_train5, x_val5, y_val5, x_test5, y_test5 = prepare_data(sequence_length, sensors, df_W_Xs_dev5, df_y_dev5, df_W_Xs_test5, df_y_test5)

x_train6, y_train6, x_val6, y_val6, x_test6, y_test6 = prepare_data(sequence_length, sensors, df_W_Xs_dev6, df_y_dev6, df_W_Xs_test6, df_y_test6)
# x_train7, y_train7, x_val7, y_val7, x_test7, y_test7 = prepare_data(sequence_length, sensors, df_W_Xs_dev7, df_y_dev7, df_W_Xs_test7, df_y_test7)
# x_train8, y_train8, x_val8, y_val8, x_test8, y_test8 = prepare_data(sequence_length, sensors, df_W_Xs_dev8, df_y_dev8, df_W_Xs_test8, df_y_test8)
# x_train9, y_train9, x_val9, y_val9, x_test9, y_test9 = prepare_data(sequence_length, sensors, df_W_Xs_dev9, df_y_dev9, df_W_Xs_test9, df_y_test9)


In [ ]:
# os.mkdir('./RULRVE_numpydata_total')

# np.save('RULRVE_numpydata_total/x_train1.npy', x_train1)
# np.save('RULRVE_numpydata_total/x_val1.npy', x_val1)
# np.save('RULRVE_numpydata_total/y_train1.npy', y_train1)
# np.save('RULRVE_numpydata_total/y_val1.npy', y_val1)
# np.save('RULRVE_numpydata_total/x_test1.npy', x_test1)
# y_test1.to_csv('RULRVE_numpydata_total/df_y_test1.csv')

# np.save('RULRVE_numpydata_total/x_train2.npy', x_train2)
# np.save('RULRVE_numpydata_total/x_val2.npy', x_val2)
# np.save('RULRVE_numpydata_total/y_train2.npy', y_train2)
# np.save('RULRVE_numpydata_total/y_val2.npy', y_val2)
# np.save('RULRVE_numpydata_total/x_test2.npy', x_test2)
# y_test2.to_csv('RULRVE_numpydata_total/df_y_test2.csv')

# np.save('RULRVE_numpydata_total/x_train3.npy', x_train3)
# np.save('RULRVE_numpydata_total/x_val3.npy', x_val3)
# np.save('RULRVE_numpydata_total/y_train3.npy', y_train3)
# np.save('RULRVE_numpydata_total/y_val3.npy', y_val3)
# np.save('RULRVE_numpydata_total/x_test3.npy', x_test3)
# y_test3.to_csv('RULRVE_numpydata_total/df_y_test3.csv')

np.save('RULRVE_numpydata_total/x_train4.npy', x_train4)
np.save('RULRVE_numpydata_total/x_val4.npy', x_val4)
np.save('RULRVE_numpydata_total/y_train4.npy', y_train4)
np.save('RULRVE_numpydata_total/y_val4.npy', y_val4)
np.save('RULRVE_numpydata_total/x_test4.npy', x_test4)
y_test4.to_csv('RULRVE_numpydata_total/df_y_test4.csv')

# np.save('RULRVE_numpydata_total/x_train5.npy', x_train5)
# np.save('RULRVE_numpydata_total/x_val5.npy', x_val5)
# np.save('RULRVE_numpydata_total/y_train5.npy', y_train5)
# np.save('RULRVE_numpydata_total/y_val5.npy', y_val5)
# np.save('RULRVE_numpydata_total/x_test5.npy', x_test5)
# y_test5.to_csv('RULRVE_numpydata_total/df_y_test5.csv')

# np.save('RULRVE_numpydata_total/x_train6.npy', x_train6)
# np.save('RULRVE_numpydata_total/x_val6.npy', x_val6)
# np.save('RULRVE_numpydata_total/y_train6.npy', y_train6)
# np.save('RULRVE_numpydata_total/y_val6.npy', y_val6)
# np.save('RULRVE_numpydata_total/x_test6.npy', x_test6)
# y_test6.to_csv('RULRVE_numpydata_total/df_y_test6.csv')

# np.save('RULRVE_numpydata_total/x_train7.npy', x_train7)
# np.save('RULRVE_numpydata_total/x_val7.npy', x_val7)
# np.save('RULRVE_numpydata_total/y_train7.npy', y_train7)
# np.save('RULRVE_numpydata_total/y_val7.npy', y_val7)
# np.save('RULRVE_numpydata_total/x_test7.npy', x_test7)
# y_test7.to_csv('RULRVE_numpydata_total/df_y_test7.csv')

# np.save('RULRVE_numpydata_total/x_train8.npy', x_train8)
# np.save('RULRVE_numpydata_total/x_val8.npy', x_val8)
# np.save('RULRVE_numpydata_total/y_train8.npy', y_train8)
# np.save('RULRVE_numpydata_total/y_val8.npy', y_val8)
# np.save('RULRVE_numpydata_total/x_test8.npy', x_test8)
# y_test8.to_csv('RULRVE_numpydata_total/df_y_test8.csv')

# np.save('RULRVE_numpydata_total/x_train9.npy', x_train9)
# np.save('RULRVE_numpydata_total/x_val9.npy', x_val9)
# np.save('RULRVE_numpydata_total/y_train9.npy', y_train9)
# np.save('RULRVE_numpydata_total/y_val9.npy', y_val9)
# np.save('RULRVE_numpydata_total/x_test9.npy', x_test9)
# y_test9.to_csv('RULRVE_numpydata_total/df_y_test9.csv')



In [ ]:
df_W_Xs_dev1.shape

In [ ]:
df_W_Xs_dev2.shape

In [ ]:
pd.concat([df_W_Xs_dev1, df_W_Xs_dev1]).shape

In [ ]:
df_y_dev1.shape

In [ ]:
df_y_dev2.shape

In [ ]:
pd.concat([df_y_dev1, df_y_dev1]).shape

In [ ]:
x_train5 = np.load('RULRVE_numpydata_total/x_train5.npy')

In [ ]:
x_train5.shape

In [ ]:
x_train5[:1000, ...].shape

In [ ]:
x_train5.shape[0]

In [ ]:
np.array_split(x_train5, 100)[-1], 

In [ ]:
x_train3 = np.load('RULRVE_numpydata_total/x_train3.npy')
y_train3 = np.load('RULRVE_numpydata_total/y_train3.npy')

In [ ]:
a = tf.data.Dataset.from_tensor_slices((x_train3, y_train3))

In [ ]:
len(list(a.as_numpy_iterator()))

In [ ]:
def loader(filepath):
    with open(filepath, 'rb') as fp:
        np_ndarray = np.load(fp)
    print("{} has a numpy ndarray of shape: {}".format(filepath, np_ndarray.shape))
    return np_ndarray

In [ ]:
from os.path import join as opj
data_dir = '/data/adas_vision_data1/users/adithya/turbofan_ncmapss/RULRVE_numpydata_total'
x_test = None
for i in [1]:#,2,3]:
    x_test = loader(opj(data_dir, 'x_test{}.npy'.format(i)))

In [ ]:
x_test

In [ ]:
for j,array in enumerate(np.array_split(x_test, 1)):
    # print("{}/100".format(j))
    print(array.shape)
    break

In [ ]:
df_y_test = pd.read_csv(opj(data_dir, 'df_y_test1.csv'))

In [ ]:
df_y_test

In [ ]:
y_test = df_y_test.RUL.values

In [ ]:
y_test.reshape(len(y_test), 1)

In [ ]:
x_test = loader(opj(data_dir, 'x_test2.npy'))

In [ ]:
x_test[1]

In [ ]:
y_val = loader(opj(data_dir, 'y_val1.npy'))

In [ ]:
x_val = loader(opj(data_dir, 'x_val1.npy'))

In [ ]:
x_train

In [ ]:
for i ,j in np.array_split(x_val, 100), y_val

# Saving the model

In [2]:
import sys
sys.path.append('/home/a0484689/PycharmProjects/Remaining-Useful-Life-Estimation-Variational')
import model

In [3]:
timesteps = 30 # x_train1.shape[1]
input_dim = 15 # x_train2.shape[2]
intermediate_dim = 300
batch_size = 1024 # 1024
latent_dim = 2
epochs = 1000 # 1000
optimizer = 'adam'
masking_value = -99
# RVE = model.create_model(timesteps, input_dim, intermediate_dim, batch_size, latent_dim, epochs, optimizer, )

In [5]:
import logging
from os.path import join as opj
data_dir = "/data/adas_vision_data1/users/adithya/turbofan_ncmapss/RULRVE_numpydata_scriptgen"
def numpy_loader(filepath):
    logger = logging.getLogger("root.numpy_loader")
    with open(filepath, 'rb') as fp:
        np_ndarray = np.load(fp)
    logger.info("{} has a numpy ndarray of shape: {}".format(filepath, np_ndarray.shape))
    return np_ndarray

filename_suffix = "DS01-005"
x_test = numpy_loader(opj(data_dir, 'x_test_{}.npy'.format(filename_suffix)))[::100, ...]
y_test = numpy_loader(opj(data_dir, 'y_test_{}.npy'.format(filename_suffix)))[::100, ...]
# test_mu = np.array([], dtype=np.float32).reshape(0, 2)
# y_test = y_test.reshape(len(y_test), 1)

In [ ]:
# ----------------------- Encoder -----------------------
inputs = Input(shape=(timesteps, input_dim,), name='encoder_input', )#batch_size=batch_size)

mask = Masking(mask_value=masking_value)(inputs)

# LSTM encoding
h = Bidirectional(LSTM(intermediate_dim))(mask) 

# VAE Z layer
mu = Dense(latent_dim)(h)
sigma = Dense(latent_dim)(h)
# z = Sampling()([mu, sigma])
epsilon = tf.keras.backend.random_normal(shape=(tf.shape(mu)[0], tf.shape(mu)[1]))
z = mu + tf.exp(0.5 * sigma) * epsilon

# Instantiate the encoder model:
encoder = keras.Model(inputs, [mu, sigma, z], name='encoder')
print(encoder.summary())
# -------------------------------------------------------


# ----------------------- Regressor --------------------
reg_latent_inputs = Input(shape=(latent_dim,), name='z_sampling_reg')
reg_intermediate = Dense(200, activation='tanh')(reg_latent_inputs)
reg_outputs = Dense(1, name='reg_output')(reg_intermediate)
# Instantiate the classifier model:
regressor = keras.Model(reg_latent_inputs, reg_outputs, name='regressor')
print(regressor.summary())
# -------------------------------------------------------


class RVE(keras.Model):
    def __init__(self, encoder, regressor, decoder=None, **kwargs):
        super(RVE, self).__init__(**kwargs)
        self.encoder = encoder
        self.regressor = regressor
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        self.reg_loss_tracker = keras.metrics.Mean(name="reg_loss")
        self.decoder = decoder
        if self.decoder!=None:
            self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
            
    def __call__(self, inputs, **kwargs):
        x = inputs
        z, _, _ = self.encoder(x)#(x, y)
        return self.regressor(z)
        
        

    @property
    def metrics(self):
        if self.decoder!=None:
            return [
                self.total_loss_tracker,
                self.kl_loss_tracker,
                self.reg_loss_tracker,
                self.reconstruction_loss_tracker
            ]
        else:
            return [
                self.total_loss_tracker,
                self.kl_loss_tracker,
                self.reg_loss_tracker,
            ]

    def train_step(self, data):
        x, target_x = data
        with tf.GradientTape() as tape:
            # kl loss
            mu, sigma, z = self.encoder(x)
            kl_loss = -0.5 * (1 + sigma - tf.square(mu) - tf.exp(sigma))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            # Regressor
            reg_prediction = self.regressor(z)
            reg_loss = tf.reduce_mean(
                keras.losses.mse(target_x, reg_prediction)
            )
            # Reconstruction
            if self.decoder!=None:
                reconstruction = self.decoder(z)
                reconstruction_loss = tf.reduce_mean(keras.losses.mse(x, reconstruction))
                total_loss = kl_loss + reg_loss + reconstruction_loss
                self.reconstruction_loss_tracker.update_state(reconstruction_loss)
            else:
                total_loss = kl_loss + reg_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        self.reg_loss_tracker.update_state(reg_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
            "reg_loss": self.reg_loss_tracker.result(),
        }


    def test_step(self, data):
        x, target_x = data

        # kl loss
        mu, sigma, z = self.encoder(x)
        kl_loss = -0.5 * (1 + sigma - tf.square(mu) - tf.exp(sigma))
        kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
        # Regressor
        reg_prediction = self.regressor(z)
        reg_loss = tf.reduce_mean(
            keras.losses.mse(target_x, reg_prediction)
        )
        # Reconstruction
        if self.decoder!=None:
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(keras.losses.mse(x, reconstruction))

            total_loss = kl_loss + reg_loss + reconstruction_loss
        else:
            total_loss = kl_loss + reg_loss

        return {
            "loss": total_loss,
            "kl_loss": kl_loss,
            "reg_loss": reg_loss,
        }

RVE = model.create_model(timesteps, input_dim, intermediate_dim,
                                 batch_size, latent_dim, epochs, optimizer, )
RVE.load_weights(tf.train.latest_checkpoint("/home/a0484689/PycharmProjects/Remaining-Useful-Life-Estimation-Variational/checkpoints_all_datasets/"))

# z, _, _ = RVE.encoder.predict(x_test[::1, ...])
# RVE.regressor.predict(z)

RVE(x_test[::100, ...])

from keras.models import Model
newInput = Input(batch_shape=(1,30,15))
newOutputs = RVE(newInput)
newModel = Model(newInput,newOutputs)

converter = tf.lite.TFLiteConverter.from_keras_model(newModel)
tfmodel = converter.convert()

open ("RVE_NCMAPSS_all_datasets_org.tflite" , "wb") .write(tfmodel)

In [ ]:
RVE = RVE(encoder, regressor)
RVE.compile(optimizer=optimizer)
RVE.load_weights(tf.train.latest_checkpoint("/home/a0484689/PycharmProjects/Remaining-Useful-Life-Estimation-Variational/checkpoints_all_datasets/"))

In [ ]:
x_test.shape

In [ ]:
y_pred = RVE(x_test[::1, ...])

In [ ]:
print("RMSE Score: {}".format(np.sqrt(mean_squared_error(y_test[::1, ...], y_pred))))
print("NASA Score: {}".format(utils.score(y_test[::1, ...], y_pred)))

converter = tf.lite.TFLiteConverter.from_keras_model(RVE.encoder)
tfmodel = converter.convert()
open ("RVE_NCMAPSS_DS01_3hidden_layers_unchanged_input_shape_encoder.tflite" , "wb") .write(tfmodel)

In [ ]:
from keras.models import Model
newInput = Input(batch_shape=(1,30,15))
newOutputs = RVE(newInput)
newModel = Model(newInput,newOutputs)

converter = tf.lite.TFLiteConverter.from_keras_model(newModel)
tfmodel = converter.convert()
open ("RVE_NCMAPSS_DS01_3hidden_layers_unchanged_input_shape.tflite" , "wb") .write(tfmodel)

In [ ]:
newOutputs